In [59]:
# Create by: Joelson Antônio dos Santos in December-16-2018.
# This implementation follows a mixture of two main bibliographic references (presented below) 

# [ref 1] - ESTER, M.; KRIEGEL, H.-P.; SANDER, J.; XU, X. A density-based algorithm for discovering
# clusters a density-based algorithm for discovering clusters in large spatial databases with noise.
# In: Proceedings of the Second International Conference on Knowledge Discovery and Data
# Mining. AAAI Press, 1996. (KDD’96), p. 226–231. Available in: <http://dl.acm.org/citation.
# cfm?id=3001460.3001507>.

#[ref 2] - CAMPELLO, R. J. G. B.; MOULAVI, D.; SANDER, J. Density-based clustering based on
# hierarchical density estimates. In: PEI, J.; TSENG, V. S.; CAO, L.; MOTODA, H.; XU, G. (Ed.).
# Advances in Knowledge Discovery and Data Mining. Berlin, Heidelberg: Springer Berlin
# Heidelberg, 2013. p. 160–172. ISBN 978-3-642-37456-2

import numpy as np
import sys
from collections import deque
from scipy.spatial import distance
from sklearn.datasets import load_iris # just for tests

class DBSCAN:
    def __init__(self, mpts, distanceFunction):
        self.setMpts(mpts)
        self.setDistanceFunction(distanceFunction)
        self.setLabels([])
    
    def setMpts(self, mpts):
        self.mpts = mpts
    def setDistanceFunction(self, distanceFunction):
        self.distanceFunction = distanceFunction
    def setLabels(self, labels):
        self.labels = labels
    def getMpts(self):
        return self.mpts
    def getDistanceFunction(self):
        return self.distanceFunction
    def getLabels(self):
        return self.labels
        
    # some metric (dis-similarities) provided by scipy library (package)
    def metric(self, p1, p2):
        if self.distanceFunction == "euclidean":
            return distance.euclidean(p1, p2)
        elif self.distanceFunction == "manhattan":
            return distance.cityblock(p1, p2)
        elif self.distanceFunction == "cosine":
            return distance.cosine(p1, p2)
        else:
            return distance.euclidean(p1, p2) # default
    
    # Heuristic method (k-NN)
    def epsEstimator(self, dataset):
        rows = len(dataset) - 1
        maxNNPoints = np.zeros(rows)
        for point in range(0, rows): #[improvement] select sample points, not all.
            kNNDistances = np.zeros(self.mpts)
            for i in range(0, self.mpts - 1):
                kNNDistances[i] = sys.float_info.max
            for neighbor in range(0, rows):
                if point == neighbor:
                    continue
                # compute distance
                distance = self.metric(dataset[point,], dataset[neighbor,]) 
                shiftLeft = len(kNNDistances) - 1
                while shiftLeft > 0 and kNNDistances[shiftLeft] > distance:
                    shiftLeft = shiftLeft - 1
                if shiftLeft == 0 and kNNDistances[shiftLeft] > distance:
                    for i in range(len(kNNDistances) - 1, 0, -1):
                        kNNDistances[i] = kNNDistances[i - 1]
                    kNNDistances[0] = distance
                elif shiftLeft == 0 and kNNDistances[shiftLeft] <= distance:
                    for i in range(len(kNNDistances) - 1, 1, -1):
                        kNNDistances[i] = kNNDistances[i - 1]
                    kNNDistances[1] = distance
                else:
                    for i in range(len(kNNDistances) - 1, shiftLeft, -1):
                        kNNDistances[i] = kNNDistances[i - 1]
                    kNNDistances[shiftLeft] = distance
            maxNNPoints[point] = kNNDistances[len(kNNDistances) - 1]
        return np.median(np.sort(maxNNPoints, kind='quicksort'))
     
    def fit(self, dataset):
        # retrieve core points
        rows = len(dataset) - 1
        eps = self.epsEstimator(dataset)
        # creating an empty adjacent list (graph representation)
        adjacentList = dict()
        for vertex in range(0, rows):
            adjacentList[vertex] = []
        for point in range(0, rows):
            for neighbor in range(point+1, rows):
                if point == neighbor:
                    continue
                # compute distance
                distance = self.metric(dataset[point,], dataset[neighbor,])
                if distance <= eps:
                    adjacentList[point].append(neighbor)
                    adjacentList[neighbor].append(point)
        # determining all points as noise, initially
        self.labels = np.zeros(rows)
        # BFS on each vertex 
        visitedVertice = np.zeros(rows)
        nextClusterLabel = 2
        for vertex in range(0, rows):
            queue = deque([vertex])
            component = [vertex]
            if visitedVertice[vertex] == 0:
                visitedVertice[vertex] = 1
            else:
                continue
            while len(queue) > 0:
                root = queue.pop()
                for adj in adjacentList[root]:
                    if visitedVertice[adj] == 0:
                        visitedVertice[adj] = 1
                        component.append(adj)
            if len(component) >= self.mpts:
                self.labels[component] = nextClusterLabel
                nextClusterLabel = nextClusterLabel + 1
            component.clear() # remove all elements 


# test method (can pass dataset[array numpy] or file name as argument)           
def runner(mpts, distanceFunction):
    # reading a dataset from file
    #dataset = np.loadtxt("iris.txt", dtype='float', delimiter=',')
    # OR loading some one from a library
    dataset = load_iris()
    model = DBSCAN(mpts, distanceFunction)
    model.fit(dataset['data'])
    # label clusters
    print("DBSCAN using", model.getDistanceFunction(), "distance: ", model.getLabels())




In [60]:
# How to run it
result1 = runner(4, "euclidean")
result2 = runner(4, "manhattan")
result3 = runner(4, "cosine")

DBSCAN using euclidean distance:  [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 3. 3. 3. 4. 3. 3. 3. 4. 3. 4. 4. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 4. 4. 4. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 4. 3. 3.
 3. 3. 4. 3. 5. 3. 3. 3. 3. 5. 4. 3. 3. 5. 3. 3. 3. 3. 3. 3. 3. 5. 5. 3.
 3. 3. 5. 3. 3. 3. 3. 3. 3. 3. 3. 5. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3.]
DBSCAN using manhattan distance:  [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 3. 3. 3. 3. 3. 3. 3. 4. 3. 3. 4. 3. 3. 3. 3. 3. 3. 3. 3. 4. 3. 3.
 3. 3. 3. 3. 3. 3. 3. 4. 4. 4. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 4. 3. 3.
 3. 3. 4. 3. 3. 3. 3. 3. 3. 5. 4. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 5. 5. 3.
 3. 3. 5. 3. 3. 3. 3. 3. 3. 3. 3. 5. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 3.]
DBSCAN using cosine di